In [1]:
import json

from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer


#Feature Selection
from sklearn.feature_selection import SelectKBest, RFE

#Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

# Cross-Validation
from sklearn.model_selection import GroupKFold

from biopsykit.classification.model_selection import SklearnPipelinePermuter

import matplotlib.pyplot as plt

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
save_results = True

In [3]:
data_path = Path("../../results/data")
data_path

WindowsPath('../../results/data')

In [4]:
models_path = Path("../../results/models")

In [5]:
input_data = pd.read_csv(data_path.joinpath("train_data.csv"), index_col=[0,1,2,3])
input_data

arbol2017-isoelectric-crossings_forouzanfar2018  \
participant phase         b_point_sample_reference heartbeat_id_reference                                                    
GDN0005     HoldingBreath 388.0                    0                                                                 438.0   
                          404.0                    1                                                                 420.0   
                          376.0                    3                                                                 382.0   
                          390.0                    4                                                                 394.0   
                          386.0                    5                                                                 398.0   
...                                                                                                                    ...   
VP_032      Talk          310.0                    39                                                                335.0   
                          322.0                    40                                                                331.0   
                          340.0                    41                                                                317.0   
                          311.0                    42                                                                365.0   
                          307.0                    43                                                                342.0   

                                                                           arbol2017-isoelectric-crossings_linear-interpolation  \
participant phase         b_point_sample_reference heartbeat_id_reference                                                         
GDN0005     HoldingBreath 388.0                    0                                                                   438.0      
                          404.0                    1                                                                   422.0      
                          376.0                    3                                                                   382.0      
                          390.0                    4                                                                   394.0      
                          386.0                    5                                                                   398.0      
...                                                                                                                      ...      
VP_032      Talk          310.0                    39                                                                  335.0      
                          322.0                    40                                                                  331.0      
                          340.0                    41                                                                  317.0      
                          311.0                    42                                                                  365.0      
                          307.0                    43                                                                  342.0      

                                                                           arbol2017-isoelectric-crossings_none  \
participant phase         b_point_sample_reference heartbeat_id_reference                                         
GDN0005     HoldingBreath 388.0                    0                                                      438.0   
                          404.0                    1                                                      340.0   
                          376.0                    3                                                      382.0   
                          390.0                    4                                                      394.0   
                          386.0           

In [6]:
print(f"Min data value: {input_data.values.min()}\nMax data value: {input_data.values.max()}")

Min data value: -84.0
Max data value: 742.0


In [7]:
columns=input_data.columns
columns

Index(['arbol2017-isoelectric-crossings_forouzanfar2018',
       'arbol2017-isoelectric-crossings_linear-interpolation',
       'arbol2017-isoelectric-crossings_none',
       'arbol2017-second-derivative_forouzanfar2018',
       'arbol2017-second-derivative_linear-interpolation',
       'arbol2017-second-derivative_none',
       'arbol2017-third-derivative_forouzanfar2018',
       'arbol2017-third-derivative_linear-interpolation',
       'arbol2017-third-derivative_none',
       'debski1993-second-derivative_forouzanfar2018',
       'debski1993-second-derivative_linear-interpolation',
       'debski1993-second-derivative_none', 'drost2022_forouzanfar2018',
       'drost2022_linear-interpolation', 'drost2022_none',
       'forounzafar2018_forouzanfar2018',
       'forounzafar2018_linear-interpolation', 'forounzafar2018_none',
       'lozano2007-linear-regression_forouzanfar2018',
       'lozano2007-linear-regression_linear-interpolation',
       'lozano2007-linear-regression_none',
    

In [8]:
negative_data = input_data[input_data.values < 0]
negative_data

arbol2017-isoelectric-crossings_forouzanfar2018  \
participant phase    b_point_sample_reference heartbeat_id_reference                                                    
GDN0007     Valsalva 448.0                    0                                                                 392.0   
                                              0                                                                 392.0   
                                              0                                                                 392.0   
VP_002      Math     209.0                    22                                                                219.0   
                                              45                                                                224.0   
...                                                                                                               ...   
VP_031      Math     201.0                    34                                                                212.0   
                     199.0                    35                                                                218.0   
                     197.0                    49                                                                206.0   
                                              50                                                                220.0   
                     202.0                    55                                                                204.0   

                                                                      arbol2017-isoelectric-crossings_linear-interpolation  \
participant phase    b_point_sample_reference heartbeat_id_reference                                                         
GDN0007     Valsalva 448.0                    0                                                                   392.0      
                                              0                                                                   392.0      
                                              0                                                                   392.0      
VP_002      Math     209.0                    22                                                                  219.0      
                                              45                                                                  224.0      
...                                                                                                                 ...      
VP_031      Math     201.0                    34                                                                  212.0      
                     199.0                    35                                                                  218.0      
                     197.0                    49                                                                  206.0      
                                              50                                                                  220.0      
                     202.0                    55                                                                  213.0      

                                                                      arbol2017-isoelectric-crossings_none  \
participant phase    b_point_sample_reference heartbeat_id_reference                                         
GDN0007     Valsalva 448.0                    0                                                      392.0   
                                              0                                                      392.0   
                                              0                                                      392.0   
VP_002      Math     209.0                    22                                                     219.0   
                                              45                                                     224.0   
...                                                                                    

### The value 600 is just a random threshold and not the computed outlier border

In [9]:
high_data = input_data[input_data.values > 600]
high_data

arbol2017-isoelectric-crossings_forouzanfar2018  \
participant phase         b_point_sample_reference heartbeat_id_reference                                                    
GDN0006     Pause         516.0                    0                                                                 526.0   
                                                   0                                                                 526.0   
                                                   0                                                                 526.0   
GDN0007     HoldingBreath 574.0                    11                                                                612.0   
                                                   11                                                                612.0   
...                                                                                                                    ...   
GDN0030     Valsalva      542.0                    18                                                                546.0   
                                                   18                                                                546.0   
                                                   18                                                                546.0   
                                                   18                                                                546.0   
VP_004      Talk          344.0                    1                                                                 298.0   

                                                                           arbol2017-isoelectric-crossings_linear-interpolation  \
participant phase         b_point_sample_reference heartbeat_id_reference                                                         
GDN0006     Pause         516.0                    0                                                                   526.0      
                                                   0                                                                   526.0      
                                                   0                                                                   526.0      
GDN0007     HoldingBreath 574.0                    11                                                                  612.0      
                                                   11                                                                  612.0      
...                                                                                                                      ...      
GDN0030     Valsalva      542.0                    18                                                                  546.0      
                                                   18                                                                  546.0      
                                                   18                                                                  546.0      
                                                   18                                                                  546.0      
VP_004      Talk          344.0                    1                                                                   334.0      

                                                                           arbol2017-isoelectric-crossings_none  \
participant phase         b_point_sample_reference heartbeat_id_reference                                         
GDN0006     Pause         516.0                    0                                                      526.0   
                                                   0                                                      526.0   
                                                   0                                                      526.0   
GDN0007     HoldingBreath 574.0                    11                                                     612.0   
                                          

In [10]:
X, y, groups, group_keys = bp.classification.utils.prepare_df_sklearn(data=input_data, label_col="b_point_sample_reference", subject_col="participant", print_summary=True)

Shape of X: (10385, 30); shape of y: (10385,); number of groups: 39, class prevalence: [ 2  1  1  4  1  1  1  1  1  2  1  1  2  1  1  2  3  2  6  3  9  4  8  7
 10  4  5 11  7 13  9 11 14 11 15  9 12 15 17 21 22 20 28 27 29 28 34 35
 44 37 37 39 30 34 46 40 50 47 41 38 23 21 26 34 34 27 33 37 26 40 20 26
 33 23 23 29 31 18 28 24 19 29 17 20 24 23 23 11 18 20 16 23 26 24 22 10
 15 14 17 14 17 20 16 13 17 17 25 16 12 19 21 18 19 19 22 17 22 19 21 24
 24 18 27 21 17 23 13 14 22 21 25 12 21 18 27 14 26 17 20 23 19  9 22 16
 24 11 18 21 25 16 31 20 23 25 26 24 27 10 36 15 36 12 36 21 29 16 32 23
 55 16 48 14 38 28 31 11 52  8 56 20 54 23 50 14 53 18 63 18 56 18 55 24
 48 21 59 14 57 11 73 19 44 18 56 12 72 17 56 14 53 16 63 18 77 12 84  9
 60 16 69  9 59  4 66 16 64  7 66  9 82  9 83  6 73  8 71 10 81 10 90  9
 69  7 68  4 74  5 71  3 61  7 78  5 77  2 82  7 73  8 64  8 61  1 81  2
 76  3 63 73  2 68  3 64  2 86  6 66  2 84  2 78  1 66  2 65 72 77  2 56
  1 71 69  1 59  1 76 49 67  1 68 63 

In [11]:
group_keys

Index(['GDN0005', 'GDN0006', 'GDN0007', 'GDN0008', 'GDN0009', 'GDN0010',
       'GDN0011', 'GDN0012', 'GDN0013', 'GDN0014', 'GDN0016', 'GDN0017',
       'GDN0018', 'GDN0019', 'GDN0020', 'GDN0021', 'GDN0022', 'GDN0023',
       'GDN0024', 'GDN0025', 'GDN0027', 'GDN0028', 'GDN0029', 'GDN0030',
       'VP_001', 'VP_002', 'VP_003', 'VP_004', 'VP_005', 'VP_020', 'VP_022',
       'VP_023', 'VP_026', 'VP_027', 'VP_028', 'VP_029', 'VP_030', 'VP_031',
       'VP_032'],
      dtype='object')

In [12]:
model_dict = {
    "scaler": {"StandardScaler": StandardScaler(), "MinMaxScaler": MinMaxScaler()},
    "reduce_dim": {"SelectKBest": SelectKBest(), "RFE": RFE(SVC(kernel="linear"))},
    "clf": {
        #"KNeighborsRegressor": KNeighborsRegressor(),
        "RandomForestRegressor": RandomForestRegressor(),
        #"HistGradientBoostingRegressor": HistGradientBoostingRegressor(),
    },
}

In [ ]:
params_dict = {
    "StandardScaler": None,
    "MinMaxScaler": None,
    "SelectKBest": {"k": [2, 4, 6, 8, 10, "all"]},
    "SVC": {"C": [0.1, 1, 10]},
    "RFE": {
        "n_features_to_select": [0.4, 0.5, 0.6],
        "step": [1,2,3],
    },
    #"KNeighborsRegressor": {
    #    "n_neighbors": [8,9,10,11,12,13,14],
    #    "weights": ["uniform", "distance"],
    #    "p": [1,2],
    #    },
    "RandomForestRegressor": {
        "n_estimators": [80, 100, 120],
        "criterion": ["squared_error", "absolute_error", "friedman_mse", "poisson"],
        #"min_samples_split": [0.0001, 0.001, 0.01, 0.1],
        #"min_samples_leaf": [0.0001, 0.001, 0.01, 0.1],
        #"max_features": ["sqrt", "log2", None],
    },
    #"HistGradientBoostingRegressor": None,
}

In [29]:
pipeline_permuter = SklearnPipelinePermuter(
    model_dict=model_dict, param_dict=params_dict
)

In [30]:
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=5)

pipeline_permuter.fit(X=X, y=y, outer_cv=outer_cv, inner_cv=inner_cv, scoring="neg_mean_absolute_error", groups=groups)

Pipeline Combinations:   0%|          | 0/4 [00:00<?, ?it/s]

### Running hyperparameter search for pipeline: (('scaler', 'StandardScaler'), ('reduce_dim', 'SelectKBest'), ('clf', 'RandomForestRegressor')) with 1 parameter grid(s):
Parameter grid #0 ({'search_method': 'grid'}): {'reduce_dim__k': [2, 4, 6, 8, 10, 'all'], 'clf__n_estimators': [80, 100, 120], 'clf__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'], 'clf__min_samples_split': [0.0001, 0.001, 0.01, 0.1], 'clf__min_samples_leaf': [0.0001, 0.001, 0.01, 0.1]}


Outer CV:   0%|          | 0/5 [00:00<?, ?it/s]

Fitting 5 folds for each of 1152 candidates, totalling 5760 fits


KeyboardInterrupt: 

In [16]:
pipeline_permuter.metric_summary()

true_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                 KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   
                RFE                 KNeighborsRegressor  [370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 334...   

                                                                                         true_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                 KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   
                RFE                 KNeighborsRegressor  [[370.0, 346.0, 374.0, 316.0, 336.0, 322.0, 33...   

                                                                                          predicted_labels  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [361.49381985572927, 360.20106003009255, 368.3...   
                RFE                 KNeighborsRegressor  [359.76247416775647, 358.6218139107078, 369.19...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [360.87555699317215, 360.12889123945575, 369.0...   
                RFE                 KNeighborsRegressor  [366.3752408791234, 361.91021063862394, 370.95...   

                                                                                    predicted_labels_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[361.49381985572927, 360.20106003009255, 368....   
                RFE                 KNeighborsRegressor  [[359.76247416775647, 358.6218139107078, 369.1...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[360.87555699317215, 360.12889123945575, 369....   
                RFE                 KNeighborsRegressor  [[366.3752408791234, 361.91021063862394, 370.9...   

                                                                                             train_indices  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RFE                 KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   
                RFE                 KNeighborsRegressor  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...   

                                                                                       train_indices_folds  \
pipeline_scaler pipeline_reduce_dim pipeline_clf                                                             
StandardScaler  SelectKBest         KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RFE                 KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
MinMaxScaler    SelectKBest         KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   
                RFE                 KNeighborsRegressor  [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13...   

                                                                                              test_indices  \
pipeline_scaler pip

In [17]:
pipeline_permuter.best_hyperparameter_pipeline()

C:\Users\sebas\Development\ResearchInternship\Code\BioPsyKit\src\biopsykit\classification\model_selection\sklearn_pipeline_permuter.py:680: FutureWarning: ['param_clf__weights', 'param_reduce_dim__k', 'params'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  .agg(["mean", "std"])


mean_test_neg_mean_absolute_error  param_clf__n_neighbors  \
outer_fold                                                              
0                                   -8.746015                      14   
1                                   -9.186139                      14   
2                                   -9.063227                      14   
3                                   -9.528919                      14   
4                                   -9.014135                      14   

            param_clf__p param_clf__weights  \
outer_fold                                    
0                      1           distance   
1                      1           distance   
2                      1           distance   
3                      1           distance   
4                      1           distance   

            param_reduce_dim__n_features_to_select  param_reduce_dim__step  \
outer_fold                                                                   
0                                              0.4                     1.0   
1                                              0.4                     1.0   
2                                              0.4                     1.0   
3                                              0.4                     1.0   
4                                              0.4                     1.0   

                                                       params  \
outer_fold                                                      
0           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
1           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
2           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
3           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   
4           {'clf__n_neighbors': 14, 'clf__p': 1, 'clf__we...   

            rank_test_neg_mean_absolute_error  \
outer_fold                                      
0                                           4   
1                                          10   
2                                           2   
3                                          14   
4                                           1   

            split0_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -7.458822   
1                                     -8.647332   
2                                     -8.684895   
3                                    -10.268598   
4                                     -7.896876   

            split1_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -9.795096   
1                                    -10.437060   
2                                     -9.300020   
3                                     -9.557538   
4                                     -9.337070   

            split2_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -8.487781   
1                                     -8.602220   
2                                     -9.658825   
3                                     -8.732567   
4                                    -10.976438   

            split3_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -8.653941   
1                                     -8.943240   
2                                     -9.254500   
3                                     -9.546388   
4                                     -9.295075   

            split4_test_neg_mean_absolute_error  \
outer_fold                                        
0                                     -9.334436   
1                                     -9.300842   
2                                     -8.417894   
3                                     -9.539503   
4                                     -7.565216   

            std_test_neg_mean_abs

In [18]:
print(f"Save results: {save_results}")

Save results: True


In [ ]:
if save_results:
    pipeline_permuter.to_pickle(models_path.joinpath("RandomForest.pkl"))